In [1]:
import xarray as xr   # xarray pacckage for netcdf io and data processing
import glob           # allow unix shell like text
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import plt
import matplotlib.dates as mdates
from scipy.signal import savgol_filter
import datetime as dt

xr.set_options(display_style="html")
%matplotlib inline

In [ ]:
# Sites, blocks, treatments

sites=["ALP1","ALP2","ALP3","ALP4","SUB1","SUB2","SUB3","SUB4","BOR1","BOR2","BOR3","BOR4"]
siteID1=["Ulvehaugen","Lavisdalen","Gudmedalen","Skjelingahaugen","Alrust","Hogsete","Rambera","Veskre","Fauske","Vikesland","Arhelleren","Ovstedalen"] 
siteID2=["Ulvhaugen","Lavisdalen","Gudmedalen","Skjellingahaugen","Alrust","Hogsete","Rambera","Veskre","Fauske","Vikesland","Arhelleren","Ovstedalen"] 
siteID3=["ULV","LAV","GUD","SKJ","ALR","HOG","RAM","VES","FAU","VIK","ARH","OVS"]


blockID=[['Ulv1','Ulv2','Ulv3','Ulv4'],
         ['Lav1','Lav2','Lav3','Lav4'],
         ['Gud12','Gud13','Gud15','Gud5'], #12 13 15 5
         ['Skj1','Skj2','Skj3','Skj4'],
         ['Alr1','Alr2','Alr3','Alr5'], #1, 2,3,5
         ['Hog1','Hog2','Hog3','Hog4'],
         ['Ram4','Ram5','Ram6','Ram8'], #4 5 6 8
         ['Ves1','Ves2','Ves3','Ves4'],
         ['Fau1','Fau2','Fau4','Fau5'], #1 2 4 5
         ['Vik2','Vik3','Vik4','Vik5'], #2, 3,4,5
         ['Arh1','Arh2','Arh3','Arh4'],       
         ['Ovs1','Ovs2','Ovs3','Ovs4']] 

treatment1="FGB" # Bareground
treatment2="GF"  # Moss only
treatment3="B"   # GRASS (Graminoid and Forbs) only

In [ ]:
# Site-level climate variables

# Read raw data
atm_obs=pd.read_csv("/home/huitang/Documents/MossLichen_testbed/data_raw/GriddedDailyClimateData2009-2019.csv", index_col=None, header=0)
atm_obs2=pd.read_csv("/home/huitang/Documents/MossLichen_testbed/data_raw/Temperature/Temperature.csv", index_col=None, header=0)
atm_obs3=pd.read_csv("/home/huitang/Documents/MossLichen_testbed/data_raw/Precipitation/Precipitation.csv", index_col=None, header=0)
atm_obs4=pd.read_csv("/home/huitang/Documents/MossLichen_testbed/data_raw/SoilMoisture/SoilMoisture.csv", index_col=None, header=0)


# Example of reading data for specific plots and treatment 
# Pay attention to the column name used for selecting corresponding data
# use ""

i=0 # 0-11. This is used for selecting site

# Air Tempearture

## gridded temperature (from MetNo)
atm_obs_sel=atm_obs[(atm_obs["siteID"]==siteID[i])
                             & (atm_obs["variable"]=="temperature")]

## temperature from logger 
atm_obs2_sel=atm_obs2[(atm_obs2["siteID"]==siteID2[i])
                            & (atm_obs2["logger"]=="temp200cm")]

# Soil temperature
atm_obs2_sel2=atm_obs2[(atm_obs2["siteID"]==siteID2[i])
                            & (atm_obs2["logger"]=="tempsoil")]

# Soil moisture
atm_obs4_sel=atm_obs4[(atm_obs4["siteID"]==siteID2[i])
                            & (atm_obs4["logger"]=="soil.moisture1")]

# gridded precipitation 
atm_obs_sel=atm_obs[(atm_obs["siteID"]==siteID[i])
                             & (atm_obs["variable"]=="precipitation")]


# Example of selecting corresponding date in python
maskt1 = (atm_obs2_sel['date'] >= '2015-1-1') & (atm_obs2_sel['date'] <= '2016-12-31')
atm_obs2_sel.loc[maskrs]['date']

In [ ]:
# Treatment-level observation data, specifically for moss

# Read raw data:
cflux=pd.read_csv("/home/huitang/Documents/MossLichen_testbed/data_raw/FunCaB_clean_Cflux_2015-2017.csv", index_col=None, header=0)
soil_moist=pd.read_csv("/home/huitang/Documents/MossLichen_testbed/data_raw/FunCaB_clean_soilMoisture_2015-2019.csv", index_col=None, header=0)
soil_temp=pd.read_csv("/home/huitang/Documents/MossLichen_testbed/data_raw/FunCaB_clean_soiltemperature_2015-2016.csv", index_col=None, header=0)

i=0 # 0-11, this is used for selecting site

# Selecting corresponding data
soil_temp_bare=soil_temp[(soil_temp["siteID"]==siteID1[i]) & (soil_temp["treatment"]==treatment1)
                             & (soil_temp["blockID"].isin(blockID[i]))]
soil_temp_moss=soil_temp[(soil_temp["siteID"]==siteID1[i]) & (soil_temp["treatment"]==treatment2)
                             & (soil_temp["blockID"].isin(blockID[i]))]
soil_temp_grass=soil_temp[(soil_temp["siteID"]==siteID1[i]) & (soil_temp["treatment"]==treatment3)
                             & (soil_temp["blockID"].isin(blockID[i]))]


soil_moist_bare=soil_moist[(soil_moist["siteID"]==siteID1[i]) & (soil_moist["treatment"]==treatment1)
                             & (soil_moist["blockID"].isin(blockID[i]))]
soil_moist_moss=soil_moist[(soil_moist["siteID"]==siteID1[i]) & (soil_moist["treatment"]==treatment2)
                             & (soil_moist["blockID"].isin(blockID[i]))]
soil_moist_grass=soil_moist[(soil_moist["siteID"]==siteID1[i]) & (soil_moist["treatment"]==treatment3)
                             & (soil_moist["blockID"].isin(blockID[i]))]

cflux_bare=cflux[(cflux["siteID"]==siteID1[i]) & (cflux["treatment"]==treatment1)
                             & (cflux["blockID"].isin(blockID[i]))]
cflux_moss=cflux[(cflux["siteID"]==siteID1[i]) & (cflux["treatment"]==treatment2)
                             & (cflux["blockID"].isin(blockID[i]))]
cflux_grass=cflux[(cflux["siteID"]==siteID1[i]) & (cflux["treatment"]==treatment3)
                             & (cflux["blockID"].isin(blockID[i]))]